<a href="https://colab.research.google.com/github/narsym/deep-learning-with-tensorflow-2.0/blob/master/MNIST_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow.keras import datasets
import numpy as np
import matplotlib.pyplot as plt

randomDim = 10

In [2]:
(X_train, _), (_, _) = datasets.mnist.load_data()
X_train = (X_train.astype(np.float32) -127.5) / 127.5

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
X_train = X_train.reshape(60000,784)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
generator = Sequential()
generator.add(layers.Dense(256, input_dim = randomDim))
generator.add(layers.LeakyReLU(0.2))
generator.add(layers.Dense(512))
generator.add(layers.LeakyReLU(0.2))
generator.add(layers.Dense(1024))
generator.add(layers.LeakyReLU(0.2))
generator.add(layers.Dense(784,activation = 'tanh'))

In [0]:
discriminator = Sequential()
discriminator.add(layers.Dense(1024,input_dim = 784))
discriminator.add(layers.LeakyReLU(0.2))
discriminator.add(layers.Dropout(0.3))
discriminator.add(layers.Dense(512))
discriminator.add(layers.LeakyReLU(0.2))
discriminator.add(layers.Dense(256))
discriminator.add(layers.LeakyReLU(0.2))
discriminator.add(layers.Dropout(0.3))
discriminator.add(layers.Dense(1, activation = 'sigmoid'))

In [0]:
discriminator.trainable = False
ganInput = layers.Input(shape = (randomDim,))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = tf.keras.models.Model(inputs = ganInput, outputs = ganOutput)

In [0]:
discriminator.compile(loss = 'binary_crossentropy', optimizer = 'adam')
gan.compile(loss = 'binary_crossentropy', optimizer = 'adam')

In [0]:
!mkdir images

In [0]:
def saveGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):    
  noise = np.random.normal(0, 1, size=[examples, randomDim])    
  generatedImages = generator.predict(noise)    
  generatedImages = generatedImages.reshape(examples, 28, 28)    
  plt.figure(figsize=figsize)    
  for i in range(generatedImages.shape[0]):        
    plt.subplot(dim[0], dim[1], i+1)        
    plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')        
    plt.axis('off')    
    plt.tight_layout()    
    plt.savefig('./images/gan_generated_image_epoch_%d.png' % epoch)

In [0]:
dlosses = []
glosses = []
def train(epochs = 1,batch_size = 128):
  batchCount = int(X_train.shape[0] / batch_size)
  print('Epochs:', epochs)
  print('Batch size:', batch_size)
  print('Batches per epoch:', batchCount)

  for e in range(1, epochs + 1):
    print ('-'*15, 'Epoch %d' % e, '-'*15)
    for _ in range(batchCount):
      noise = np.random.normal(0, 1, size = [batch_size, randomDim])
      imageBatch = X_train[np.random.randint(0, X_train.shape[0], size = batch_size)]
      generatedImages = generator.predict(noise)
      X = np.concatenate([imageBatch, generatedImages])

      yDis = np.zeros(2 * batch_size)
      yDis[:batch_size] = 0.9
      discriminator.trainable = True
      dloss = discriminator.train_on_batch(X, yDis)

      noise = np.random.normal(0, 1, size = [batch_size, randomDim])
      yGen = np.ones(batch_size)
      discriminator.trainable = False
      gloss = gan.train_on_batch(noise, yGen)

      dlosses.append(dloss)
      glosses.append(gloss)

      if e == 1 or e % 20 == 0:            
        saveGeneratedImages(e)

train()

In [0]:
def plotLoss(epoch):    
  plt.figure(figsize=(10, 8))    
  plt.plot(dLosses, label='Discriminitive loss')    
  plt.plot(gLosses, label='Generative loss')    
  plt.xlabel('Epoch')    
  plt.ylabel('Loss')    
  plt.legend()    
  plt.savefig('images/gan_loss_epoch_%d.png' % epoch)